In [1]:
import pandas as pd

from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

from openai import OpenAI

## 1. Data Cleaning

In [2]:
df = pd.read_csv('../../top_rated_wines.csv')
df

,name,region,variety,rating,notes
0,3 Rings Reserve Shiraz 2004,"Barossa Valley, Barossa, South Australia, Aust...",Red Wine,96.0,Vintage Comments : Classic Barossa vintage con...
1,Abreu Vineyards Cappella 2007,"Napa Valley, California",Red Wine,96.0,Cappella is a proprietary blend of two clones ...
2,Abreu Vineyards Cappella 2010,"Napa Valley, California",Red Wine,98.0,Cappella is one of the oldest vineyard sites i...
3,Abreu Vineyards Howell Mountain 2008,"Howell Mountain, Napa Valley, California",Red Wine,96.0,When David purchased this Howell Mountain prop...
4,Abreu Vineyards Howell Mountain 2009,"Howell Mountain, Napa Valley, California",Red Wine,98.0,"As a set of wines, it is hard to surpass the f..."
...,...,...,...,...,...
1360,Lewis Cellars Alec's Blend Red 2002,"Napa Valley, California",Red Wine,96.0,Number 12 on
1361,Lewis Cellars Cabernet Sauvignon 2002,"Napa Valley, California",Red Wine,96.0,Showcasing the unique personalities of small h...
1362,Lewis Cellars Cuvee L Cabernet Sauvignon 2015,"Napa Valley, California",Red Wine,96.0,"Straight from James Fenimore Cooper’s novel, L..."
1363,Lewis Cellars Reserve Cabernet Sauvignon 2010,"Napa Valley, California",Red Wine,96.0,


In [3]:
df = df[df["variety"].notna()]
data = df.to_dict("records")
df

,name,region,variety,rating,notes
0,3 Rings Reserve Shiraz 2004,"Barossa Valley, Barossa, South Australia, Aust...",Red Wine,96.0,Vintage Comments : Classic Barossa vintage con...
1,Abreu Vineyards Cappella 2007,"Napa Valley, California",Red Wine,96.0,Cappella is a proprietary blend of two clones ...
2,Abreu Vineyards Cappella 2010,"Napa Valley, California",Red Wine,98.0,Cappella is one of the oldest vineyard sites i...
3,Abreu Vineyards Howell Mountain 2008,"Howell Mountain, Napa Valley, California",Red Wine,96.0,When David purchased this Howell Mountain prop...
4,Abreu Vineyards Howell Mountain 2009,"Howell Mountain, Napa Valley, California",Red Wine,98.0,"As a set of wines, it is hard to surpass the f..."
...,...,...,...,...,...
1360,Lewis Cellars Alec's Blend Red 2002,"Napa Valley, California",Red Wine,96.0,Number 12 on
1361,Lewis Cellars Cabernet Sauvignon 2002,"Napa Valley, California",Red Wine,96.0,Showcasing the unique personalities of small h...
1362,Lewis Cellars Cuvee L Cabernet Sauvignon 2015,"Napa Valley, California",Red Wine,96.0,"Straight from James Fenimore Cooper’s novel, L..."
1363,Lewis Cellars Reserve Cabernet Sauvignon 2010,"Napa Valley, California",Red Wine,96.0,


## 2. Construct embeddings

In [4]:
encoder = SentenceTransformer("all-MiniLM-L6-v2") # model to create embeddings
# create the vector database client
qdrant = QdrantClient(":memory:") # create in-memory Qdrant instance

ConnectionError: (ProtocolError('Connection aborted.', OSError(0, 'Error')), '(Request ID: 8706de21-38db-4c3f-a1d9-414e6bc0dcdb)')

In [16]:
# create collections to store books
qdrant.recreate_collection(
    collection_name="top_wines",
    vectors_config=models.VectorParams(size=encoder.get_sentence_embedding_dimension(), # vector size is defined by used model
                                       distance=models.Distance.COSINE)
)

NameError: name 'qdrant' is not defined

In [ ]:
# vectorize
qdrant.upload_points(
    collection_name="top_wines",
    points=[
        models.PointStruct(
            id=idx,
            vector=encoder.encode(doc["notes"]).tolist(),
            payload=doc
        )
        for idx, doc in enumerate(data) # data is the variable holding all the wines
    ]
)

In [ ]:
user_prompt = "Suggest me an amazing Malbec wine from Argentina"

In [ ]:
# search time for awesone wines
hits = qdrant.search(
    collection_name="top_wines",
    query_vector=encoder.encode("99 points Cabernet Sauvignon from Napa Valley").tolist(),
    limit=3
)

for hit in hits:
    print(hit.payload, "score:", hit.score)

In [ ]:
search_results = [hit.payload for hit in hits]

In [23]:
client = OpenAI(
    base_url="http://127.0.0.1:8080/v1",
    api_key="sk-no-key-required"
)



In [ ]:
completion = client.chat.completions.create(
    model="LLaMA_CPP",
    messages={
        {"role": "message", "content": "Content 1"},
        {"role": "user", "content": "Content 2"},
        {"role": "assistant", "content": str(search_results)}
    }
)
print(completion.choices[0].message)